# EBAC - Regressão II - regressão múltipla

## Tarefa II

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

In [3]:
# Ler a base
df = pd.read_csv('previsao_de_renda.csv')

# Remover linhas com valores faltantes
df = df.dropna()

# Criar log da renda (variável resposta do modelo)
df['log_renda'] = np.log(df['renda'])

# Remover colunas que não serão usadas como preditoras
df = df.drop(columns=['data_ref', 'id_cliente', 'Unnamed: 0'])

# Separar a variável resposta (y)
y = df['log_renda']

# Selecionar as covariáveis
X = df.drop(columns=['renda', 'log_renda'])

# Criar dummies para as variáveis qualitativas
X_dummies = pd.get_dummies(X, drop_first=True)

X_dummies.head()




,posse_de_veiculo,posse_de_imovel,qtd_filhos,idade,tempo_emprego,qt_pessoas_residencia,sexo_M,tipo_renda_Bolsista,tipo_renda_Empresário,tipo_renda_Pensionista,...,educacao_Superior incompleto,estado_civil_Separado,estado_civil_Solteiro,estado_civil_União,estado_civil_Viúvo,tipo_residencia_Casa,tipo_residencia_Com os pais,tipo_residencia_Comunitário,tipo_residencia_Estúdio,tipo_residencia_Governamental
0,False,True,0,26,6.602740,1.0,False,False,True,False,...,False,False,True,False,False,True,False,False,False,False
1,True,True,0,28,7.183562,2.0,True,False,False,False,...,False,False,False,False,False,True,False,False,False,False
2,True,True,0,35,0.838356,2.0,False,False,True,False,...,False,False,False,False,False,True,False,False,False,False
3,False,True,1,30,4.846575,3.0,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
4,True,False,0,33,4.293151,1.0,True,False,False,False,...,False,False,True,False,False,False,False,False,False,True


1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [4]:
# 1) Separar base em treino e teste (25% teste, 75% treino)
X_train, X_test, y_train, y_test = train_test_split(
    X_dummies,
    y,
    test_size=0.25,
    random_state=42
)

# Ver formatos só pra conferir
X_train.shape, X_test.shape

((9320, 24), (3107, 24))

In [5]:
# 2) Rodar regressão Ridge para vários valores de alpha

# Lista de alphas informada no enunciado
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]

# Lista para guardar os resultados
resultados_ridge = []

for alpha in alphas:
    # Pipeline com padronização + Ridge
    modelo_ridge = Pipeline([
        ('escala', StandardScaler()),
        ('ridge', Ridge(alpha=alpha))
    ])
    
    # Ajustar modelo no treino
    modelo_ridge.fit(X_train, y_train)
    
    # R² no treino e no teste
    r2_treino = modelo_ridge.score(X_train, y_train)
    r2_teste  = modelo_ridge.score(X_test, y_test)
    
    # Guardar resultados
    resultados_ridge.append({
        'alpha': alpha,
        'R2_treino': r2_treino,
        'R2_teste': r2_teste
    })

# Colocar tudo em um DataFrame para visualizar
df_ridge = pd.DataFrame(resultados_ridge)
df_ridge



,alpha,R2_treino,R2_teste
0,0.000,0.354405,0.364486
1,0.001,0.354405,0.364486
2,0.005,0.354405,0.364486
3,0.010,0.354405,0.364486
4,0.050,0.354405,0.364486
5,0.100,0.354405,0.364487


In [6]:
# Descobrir qual alpha deu o melhor R² na base de teste
melhor_ridge = df_ridge.loc[df_ridge['R2_teste'].idxmax()]
melhor_ridge_alpha = melhor_ridge['alpha']
melhor_ridge


alpha        0.100000
R2_treino    0.354405
R2_teste     0.364487
Name: 5, dtype: float64

In [13]:
# 3) Rodar regressão LASSO para os alphas (sem alpha = 0)

# Lista de alphas só com valores > 0
alphas_lasso = [0.001, 0.005, 0.01, 0.05, 0.1]

resultados_lasso = []

for alpha in alphas_lasso:
    # Ajuste do modelo LASSO com padronização
    modelo_lasso = Pipeline([
        ('escala', StandardScaler()),
        ('lasso', Lasso(alpha=alpha, max_iter=10000))
    ])
    
    # Treinar no conjunto de treino
    modelo_lasso.fit(X_train, y_train)
    
    # R² no treino e no teste
    r2_treino = modelo_lasso.score(X_train, y_train)
    r2_teste  = modelo_lasso.score(X_test, y_test)
    
    resultados_lasso.append({
        'alpha': alpha,
        'R2_treino': r2_treino,
        'R2_teste': r2_teste
    })

df_lasso = pd.DataFrame(resultados_lasso)
df_lasso


,alpha,R2_treino,R2_teste
0,0.001,0.353828,0.365187
1,0.005,0.353183,0.365076
2,0.010,0.351784,0.364206
3,0.050,0.331657,0.344569
4,0.100,0.307619,0.316976


In [8]:
# Melhor alpha do LASSO com base no R² de teste
melhor_lasso = df_lasso.loc[df_lasso['R2_teste'].idxmax()]
melhor_lasso_alpha = melhor_lasso['alpha']
melhor_lasso


alpha        0.001000
R2_treino    0.353828
R2_teste     0.365187
Name: 1, dtype: float64

In [9]:
# 4) Stepwise (seleção forward) usando Regressão Linear

# Função de stepwise forward
def stepwise_forward(X_train, y_train, X_test, y_test):
    escolhidas = []
    restantes = list(X_train.columns)
    melhor_r2 = -np.inf
    
    while len(restantes) > 0:
        candidatos = []
        
        for coluna in restantes:
            cols_teste = escolhidas + [coluna]
            
            # Modelo linear simples
            modelo = LinearRegression()
            modelo.fit(X_train[cols_teste], y_train)
            
            y_pred = modelo.predict(X_test[cols_teste])
            r2 = r2_score(y_test, y_pred)
            
            candidatos.append((coluna, r2))
        
        # Melhor coluna deste ciclo
        melhor_coluna, melhor_r2_ciclo = max(candidatos, key=lambda x: x[1])
        
        # Se não melhorar mais, para
        if melhor_r2_ciclo <= melhor_r2:
            break
        
        # Atualiza lista de escolhidas
        escolhidas.append(melhor_coluna)
        restantes.remove(melhor_coluna)
        melhor_r2 = melhor_r2_ciclo
    
    return escolhidas, melhor_r2

# Rodar o stepwise
colunas_stepwise, r2_stepwise = stepwise_forward(X_train, y_train, X_test, y_test)

colunas_stepwise, r2_stepwise


(['tempo_emprego',
  'sexo_M',
  'educacao_Superior completo',
  'tipo_renda_Empresário',
  'idade',
  'posse_de_veiculo',
  'posse_de_imovel',
  'qt_pessoas_residencia',
  'estado_civil_Separado',
  'tipo_renda_Servidor público',
  'educacao_Superior incompleto',
  'tipo_residencia_Estúdio',
  'estado_civil_União',
  'educacao_Secundário',
  'tipo_renda_Pensionista',
  'tipo_renda_Bolsista',
  'estado_civil_Solteiro'],
 0.36630628779179986)

In [10]:
# Organizar os melhores resultados em um dicionário para facilitar a comparação

comparacao_modelos = {
    'Ridge': {
        'melhor_alpha': melhor_ridge_alpha,
        'R2_teste': melhor_ridge['R2_teste']
    },
    'Lasso': {
        'melhor_alpha': melhor_lasso_alpha,
        'R2_teste': melhor_lasso['R2_teste']
    },
    'Stepwise': {
        'variaveis_escolhidas': colunas_stepwise,
        'R2_teste': r2_stepwise
    }
}

comparacao_modelos


{'Ridge': {'melhor_alpha': 0.1, 'R2_teste': 0.36448696765268784},
 'Lasso': {'melhor_alpha': 0.001, 'R2_teste': 0.36518693942640335},
 'Stepwise': {'variaveis_escolhidas': ['tempo_emprego',
   'sexo_M',
   'educacao_Superior completo',
   'tipo_renda_Empresário',
   'idade',
   'posse_de_veiculo',
   'posse_de_imovel',
   'qt_pessoas_residencia',
   'estado_civil_Separado',
   'tipo_renda_Servidor público',
   'educacao_Superior incompleto',
   'tipo_residencia_Estúdio',
   'estado_civil_União',
   'educacao_Secundário',
   'tipo_renda_Pensionista',
   'tipo_renda_Bolsista',
   'estado_civil_Solteiro'],
  'R2_teste': 0.36630628779179986}}

In [11]:
# 6) Criar novas variáveis para tentar melhorar o R²

X_transf = X.copy()

# Ajuste de variáveis: criar idade ao quadrado
X_transf['idade2'] = X_transf['idade'] ** 2

# Ajuste de variáveis: log(tempo_emprego + 1) para suavizar escala
X_transf['tempo_emprego_log'] = np.log1p(X_transf['tempo_emprego'])

# Criar dummies novamente
X_transf_dummies = pd.get_dummies(X_transf, drop_first=True)

# Separar treino e teste com as novas variáveis (mesmo random_state)
X_train_t, X_test_t, y_train_t, y_test_t = train_test_split(
    X_transf_dummies,
    y,
    test_size=0.25,
    random_state=42
)

# Reajustar Ridge usando o melhor alpha encontrado antes
modelo_ridge_transf = Pipeline([
    ('escala', StandardScaler()),
    ('ridge', Ridge(alpha=melhor_ridge_alpha))
])

modelo_ridge_transf.fit(X_train_t, y_train_t)

r2_teste_transf = modelo_ridge_transf.score(X_test_t, y_test_t)
r2_teste_transf


0.3670961963028533

In [12]:
# 7) Ajustar uma árvore de regressão

modelo_arvore = DecisionTreeRegressor(
    max_depth=5,          # profundidade máxima (pode testar outros valores)
    min_samples_leaf=50,  # mínimo de amostras em cada folha
    random_state=42
)

# Ajustar no mesmo X_train / y_train original
modelo_arvore.fit(X_train, y_train)

# R² no treino e no teste
r2_arvore_treino = modelo_arvore.score(X_train, y_train)
r2_arvore_teste  = modelo_arvore.score(X_test, y_test)

r2_arvore_treino, r2_arvore_teste


(0.37895147032578236, 0.37398504794954646)

Após testar diferentes modelos, observei que Ridge e LASSO apresentaram desempenhos muito próximos, com R² em torno de 0.36. O modelo Stepwise teve um ganho pequeno ao selecionar apenas as variáveis mais relevantes, e as transformações aplicadas geraram uma leve melhoria no ajuste. Mesmo assim, o melhor resultado foi obtido pela Árvore de Regressão, que apresentou o maior R² na base de testes e conseguiu representar relações não lineares que os modelos lineares não capturam. Assim, concluo que, para esta base de dados, a Árvore de Regressão foi a técnica com melhor capacidade preditiva.